In [1]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v6.0 - PERSISTENT MODE
Conexiones más largas para maximizar interacción
"""

import hashlib
import struct
import socket
import time
import random
import select
from typing import List, Tuple, Dict
from dataclasses import dataclass
from collections import defaultdict
from datetime import datetime

@dataclass
class BlockHeader:
    version: int
    prev_hash: bytes
    merkle_root: bytes
    timestamp: int
    bits: int
    nonce: int

    def serialize(self) -> bytes:
        return (struct.pack('<I', self.version) + self.prev_hash +
                self.merkle_root + struct.pack('<I', self.timestamp) +
                struct.pack('<I', self.bits) + struct.pack('<I', self.nonce))

    def hash(self) -> bytes:
        return hashlib.sha256(hashlib.sha256(self.serialize()).digest()).digest()

    def hash_hex(self) -> str:
        return self.hash()[::-1].hex()

@dataclass
class Block:
    header: BlockHeader
    coinbase_script: bytes = b''

    def serialize(self) -> bytes:
        data = self.header.serialize()
        data += b'\x01'
        data += self._coinbase()
        return data

    def _coinbase(self) -> bytes:
        tx = struct.pack('<I', 1)
        tx += b'\x01' + bytes(32) + struct.pack('<I', 0xffffffff)
        script = self.coinbase_script or b'\x04KAORU'
        tx += bytes([len(script)]) + script + struct.pack('<I', 0xffffffff)
        tx += b'\x01' + struct.pack('<Q', 50 * 10**8)
        tx += b'\x19\x76\xa9\x14' + bytes(20) + b'\x88\xac'
        tx += struct.pack('<I', 0)
        return tx


class KaoruBridgeV6:
    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    MSG_BLOCK = 2

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.our_blocks: List[Block] = []
        self.stats = defaultdict(int)
        self.received_invs = []
        self.conversation_log = []

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    def log_conv(self, direction: str, ip: str, msg: str):
        timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
        self.conversation_log.append(f"[{timestamp}] {direction} {ip}: {msg}")

    def create_block(self, height: int, prev_hash: bytes) -> Block:
        script = bytes([0x01, height]) + f"KAORU_v6_{height}".encode()
        merkle = self.double_sha256(script)
        header = BlockHeader(0x20000000, prev_hash, merkle, int(time.time()), 0x1d00ffff, 0)
        return Block(header=header, coinbase_script=script)

    def mine_block(self, block: Block, target_zeros: int = 4) -> bool:
        target = "0" * target_zeros
        for nonce in range(50_000_000):
            block.header.nonce = nonce
            if block.header.hash_hex().startswith(target):
                return True
        return False

    def build_chain(self, n: int = 3, zeros: int = 4):
        print(f"\n   ⛏️ Minando {n} bloques...")
        prev = bytes(32)
        for i in range(n):
            block = self.create_block(i, prev)
            if self.mine_block(block, zeros):
                self.our_blocks.append(block)
                prev = block.header.hash()
                print(f"      [{i}] {block.header.hash_hex()[:32]}... ✅")

    def msg(self, cmd: str, payload: bytes) -> bytes:
        c = cmd.encode().ljust(12, b'\x00')
        return self.magic + c + struct.pack('<I', len(payload)) + self.double_sha256(payload)[:4] + payload

    def version_payload(self, ip: str) -> bytes:
        p = struct.pack('<i', 70016) + struct.pack('<Q', 1037) + struct.pack('<q', int(time.time()))
        p += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', 8333)
        p += struct.pack('<Q', 1037) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', 8333)
        p += struct.pack('<Q', random.randint(0, 2**64-1))
        p += b'\x10/KaoruBridge:6.0/'
        p += struct.pack('<i', len(self.our_blocks)) + b'\x01'
        return p

    def inv_payload(self) -> bytes:
        p = bytes([len(self.our_blocks)])
        for b in self.our_blocks:
            p += struct.pack('<I', self.MSG_BLOCK) + b.header.hash()
        return p

    def headers_payload(self) -> bytes:
        p = bytes([len(self.our_blocks)])
        for b in self.our_blocks:
            p += b.header.serialize() + b'\x00'
        return p

    def parse_msgs(self, data: bytes) -> List[Tuple[str, bytes]]:
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except:
                break
        return msgs

    def recv_with_timeout(self, sock: socket.socket, timeout: float) -> bytes:
        sock.setblocking(False)
        data = b''
        end = time.time() + timeout
        while time.time() < end:
            try:
                r, _, _ = select.select([sock], [], [], 0.1)
                if r:
                    chunk = sock.recv(16384)
                    if chunk:
                        data += chunk
                    else:
                        break
            except:
                break
        sock.setblocking(True)
        return data

    def persistent_session(self, ip: str, duration: int = 30) -> Dict:
        """Sesión persistente con un nodo."""

        result = {
            'ip': ip,
            'connected': False,
            'duration': 0,
            'messages': defaultdict(int),
            'headers_sent': 0,
            'blocks_sent': 0,
            'blocks_requested': False,
            'peer_info': '',
            'events': []
        }

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(10)
            sock.connect((ip, 8333))
            result['connected'] = True
            start_time = time.time()

            # Handshake
            sock.send(self.msg('version', self.version_payload(ip)))
            result['messages']['sent_version'] += 1
            result['events'].append("→ VERSION")

            time.sleep(0.5)

            # Loop principal
            while time.time() - start_time < duration:
                data = self.recv_with_timeout(sock, 2.0)

                if not data:
                    time.sleep(0.3)
                    continue

                for cmd, payload in self.parse_msgs(data):
                    result['messages'][f'recv_{cmd}'] += 1
                    self.stats[f'recv_{cmd}'] += 1

                    if cmd == 'version':
                        try:
                            ua_len = payload[80]
                            result['peer_info'] = payload[81:81+ua_len].decode('ascii', errors='ignore')
                        except:
                            pass

                        sock.send(self.msg('verack', b''))
                        result['messages']['sent_verack'] += 1
                        result['events'].append("→ VERACK")

                        # Enviar INV
                        time.sleep(0.2)
                        sock.send(self.msg('inv', self.inv_payload()))
                        result['messages']['sent_inv'] += 1
                        result['events'].append(f"→ INV ({len(self.our_blocks)} bloques)")

                    elif cmd == 'verack':
                        result['events'].append("← VERACK")

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload))
                        result['messages']['sent_pong'] += 1

                    elif cmd == 'sendcmpct':
                        sock.send(self.msg('sendcmpct', b'\x00' + struct.pack('<Q', 2)))
                        result['messages']['sent_sendcmpct'] += 1

                    elif cmd == 'getheaders':
                        result['events'].append("← GETHEADERS ⭐")
                        print(f"      🎯 GETHEADERS recibido!")

                        sock.send(self.msg('headers', self.headers_payload()))
                        result['messages']['sent_headers'] += 1
                        result['headers_sent'] += 1
                        result['events'].append(f"→ HEADERS ({len(self.our_blocks)})")
                        print(f"      📤 HEADERS enviados: {len(self.our_blocks)}")

                    elif cmd == 'getdata':
                        result['blocks_requested'] = True
                        result['events'].append("← GETDATA ⭐⭐")
                        print(f"      🔥 GETDATA recibido!")

                        count = payload[0] if payload else 0
                        offset = 1

                        for _ in range(count):
                            if offset + 36 > len(payload):
                                break
                            inv_type = struct.unpack('<I', payload[offset:offset+4])[0]
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36

                            if inv_type == self.MSG_BLOCK:
                                for block in self.our_blocks:
                                    if block.header.hash() == inv_hash:
                                        print(f"      📦 ENVIANDO BLOQUE: {block.header.hash_hex()[:24]}...")
                                        sock.send(self.msg('block', block.serialize()))
                                        result['messages']['sent_block'] += 1
                                        result['blocks_sent'] += 1
                                        result['events'].append(f"→ BLOCK {block.header.hash_hex()[:16]}")
                                        self.stats['sent_block'] += 1

                    elif cmd == 'inv':
                        count = payload[0] if payload else 0
                        result['events'].append(f"← INV ({count} items)")
                        self.received_invs.append((ip, count))

                    elif cmd == 'reject':
                        result['events'].append(f"← REJECT ❌")
                        print(f"      ❌ REJECT recibido")

                    elif cmd == 'headers':
                        count = payload[0] if payload else 0
                        result['events'].append(f"← HEADERS ({count})")
                        if count > 0:
                            print(f"      📥 Recibidos {count} headers del nodo")

            result['duration'] = time.time() - start_time
            sock.close()

        except Exception as e:
            result['error'] = str(e)

        return result

    def discover_nodes(self) -> List[str]:
        nodes = []
        for seed in ['seed.bitcoin.sipa.be', 'dnsseed.bluematt.me', 'seed.bitcoinstats.com']:
            try:
                nodes.extend(socket.gethostbyname_ex(seed)[2][:6])
            except:
                pass
        random.shuffle(nodes)
        return nodes

    def execute(self, num_blocks: int = 3, target_zeros: int = 4,
                num_peers: int = 5, session_duration: int = 20):

        print("""
╔════════════════════════════════════════════════════════════════════════════════╗
║                                                                                ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██████╗            ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║██╔════╝            ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║███████╗            ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝██╔═══██╗           ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝ ╚██████╔╝           ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═════╝            ║
║                                                                                ║
║                        PERSISTENT CONNECTION MODE                              ║
║                        Session Duration: {session_duration}s per peer                         ║
║                                                                                ║
╚════════════════════════════════════════════════════════════════════════════════╝
        """)

        # Construir cadena
        print("=" * 80)
        print("[FASE 1] ⛏️  MINANDO BLOCKCHAIN")
        print("=" * 80)
        self.build_chain(num_blocks, target_zeros)

        # Descubrir nodos
        print("\n" + "=" * 80)
        print("[FASE 2] 🌐 DESCUBRIENDO NODOS")
        print("=" * 80)
        nodes = self.discover_nodes()
        print(f"\n   Encontrados: {len(nodes)} nodos")

        # Sesiones persistentes
        print("\n" + "=" * 80)
        print(f"[FASE 3] 📡 SESIONES PERSISTENTES ({session_duration}s c/u)")
        print("=" * 80)

        results = []

        for ip in nodes[:num_peers]:
            print(f"\n   ╔══ 📡 {ip} ══╗")
            print(f"   ║  Iniciando sesión de {session_duration}s...")

            result = self.persistent_session(ip, session_duration)
            results.append(result)

            if result['connected']:
                print(f"   ║  👤 {result['peer_info'][:35]}")
                print(f"   ║  ⏱️  Duración: {result['duration']:.1f}s")
                print(f"   ║  📤 Headers enviados: {result['headers_sent']}")
                print(f"   ║  📦 Bloques enviados: {result['blocks_sent']}")
                print(f"   ║  🎯 Bloques solicitados: {'✅ SÍ!' if result['blocks_requested'] else '❌'}")

                print(f"   ║  📋 Eventos:")
                for event in result['events'][-8:]:
                    print(f"   ║     {event}")

                print(f"   ╚══ {'✅ ÉXITO' if result['headers_sent'] > 0 else '⚠️ PARCIAL'} ══╝")
            else:
                print(f"   ╚══ ❌ {result.get('error', 'Error')[:30]} ══╝")

        # Resultados
        print("\n" + "=" * 80)
        print("                         📊 RESULTADOS FINALES")
        print("=" * 80)

        connected = sum(1 for r in results if r['connected'])
        headers = sum(r['headers_sent'] for r in results)
        blocks = sum(r['blocks_sent'] for r in results)
        requested = sum(1 for r in results if r['blocks_requested'])

        print(f"""
   ╔═══════════════════════════════════════════════════════════════════════════╗
   ║                              ESTADÍSTICAS                                 ║
   ╠═══════════════════════════════════════════════════════════════════════════╣
   ║                                                                           ║
   ║   Bloques en cadena:        {len(self.our_blocks):3d}                                         ║
   ║   Peers contactados:        {num_peers:3d}                                         ║
   ║   Conexiones exitosas:      {connected:3d}                                         ║
   ║   Headers enviados:         {headers:3d}                                         ║
   ║   Bloques solicitados por:  {requested:3d} peers                                   ║
   ║   Bloques enviados:         {blocks:3d}                                         ║
   ║                                                                           ║
   ╚═══════════════════════════════════════════════════════════════════════════╝
        """)

        print("   📊 Mensajes totales:")
        for key in sorted(self.stats.keys()):
            d = "📤" if 'sent' in key else "📥"
            print(f"      {d} {key.replace('recv_','').replace('sent_',''):15s}: {self.stats[key]}")

        if self.received_invs:
            print(f"\n   📥 INVs recibidos de peers:")
            for ip, count in self.received_invs[:5]:
                print(f"      {ip}: {count} items")

        print(f"""
   ╔═══════════════════════════════════════════════════════════════════════════╗
   ║                                                                           ║
   ║   🏆 LOGROS v6.0:                                                         ║
   ║                                                                           ║
   ║   {'✅' if len(self.our_blocks) > 0 else '❌'} Blockchain minada con PoW                                     ║
   ║   {'✅' if connected > 0 else '❌'} Conexiones persistentes a mainnet                            ║
   ║   {'✅' if headers > 0 else '❌'} Headers enviados a {headers} peers                                   ║
   ║   {'✅' if requested > 0 else '❌'} Peers solicitaron GETDATA                                    ║
   ║   {'✅' if blocks > 0 else '❌'} Bloques completos propagados                                   ║
   ║                                                                           ║
   ╚═══════════════════════════════════════════════════════════════════════════╝

   Estado: KAORU BRIDGE v6.0 COMPLETADO ✅
        """)

        return results


if __name__ == "__main__":
    bridge = KaoruBridgeV6()
    bridge.execute(num_blocks=3, target_zeros=4, num_peers=5, session_duration=25)


╔════════════════════════════════════════════════════════════════════════════════╗
║                                                                                ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██████╗            ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║██╔════╝            ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║███████╗            ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝██╔═══██╗           ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝ ╚██████╔╝           ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═════╝            ║
║                                                                                ║
║                        PERSISTENT CONNECTION MODE                              ║
║                        Session Duration: {session_duration}s per peer                         ║
║                                                                            

no  le hagan caso a las notas XD